In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.SparkSession
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

In [ ]:
val df3 = spark.read.format("jdbc").table("houses")

In [ ]:
df3.show()